# RQ1: Which technologies have been investigated in the last decade?

## Module importieren

In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity
from spacy.symbols import NOUN
from spacy.matcher import Matcher
from spacy.util import filter_spans

## Daten einlesen
In diesem Block werden die Daten in eine Pandas DataFrame geladen. Anschließend werden die Spalten zur besseren Übersichtlichkeit umbenannt.

In [2]:
#Datene einlesen
data = pd.read_csv('./data/data.csv',usecols = ['Unnamed: 0','Document Title','Abstract','Publication Year'])
data.columns = ['index','title','year','abstract']
data.index = data['index']

data.drop(["index"], axis = 1, inplace = True)

data.head()

data.groupby('year').count()

,title,abstract
year,,
2009,139,139
2010,132,131
2011,114,114
2012,159,159
2013,149,149
2014,137,137
2015,171,171
2016,186,186
2017,174,174


## Funktion für die TermDokumentenMatrix

In [3]:
def CreateTermDocMatrix(column):
        
    count_vectorizer = CountVectorizer(stop_words='english',ngram_range = (1,2),dtype=np.int32)
    sparse_matrix = count_vectorizer.fit_transform(data[column].values.astype('U'))

    doc_term_matrix = sparse_matrix.todense()
    df = pd.DataFrame(doc_term_matrix, columns=count_vectorizer.get_feature_names(), dtype=np.int32)
    df['year'] = data['year']
    returndf = df.groupby('year').sum().transpose().astype('int32')
    return returndf

In [4]:
def ShowWord(data,word):
    d = data.loc[word]
    d.plot.bar()

In [5]:
def SpaltenEntf(data):
    y = []
    for item in data.columns:
        y = data.columns


    for index in range(len(y)):
        x = y[index]
        if x in stopwords or x.isnumeric():
            del data[y[index]]
    return data

## Erstellen der TermDokumentenmatrix

In [6]:
def Retokenize(text):
    # Verarbeite den Text
    if type(text) is str:
        doc = nlp(text)


    matches = matcher(doc)
    sp  = [doc[match[1]:match[2]] for match in matches] 

    filtered = filter_spans(sp)
    with doc.retokenize() as retokenizer:
        for s in filtered:
            retokenizer.merge(s, attrs = {"POS": "NOUN"})
    [(token.pos_, token.text) for token in doc]
        
    #Returns spacy doc
    return doc
                
        

In [7]:
from matplotlib import pyplot as plt
years = []

#Balkendiagramm
#plt.bar(range(len(years)),anzah)

#Titel
#plt.title("Toller Plot")
#plt.ylabel("# of awards")

#plt.xticks(range(len(years)),years)
#plt.show()

## NLP Model laden

In [8]:
nlp = spacy.load("en_core_web_sm")

## Counter und Matcher initialisieren

In [9]:
matcher = Matcher(nlp.vocab)
words = Counter()
words_chunk = Counter()

## Pattern festlegen und dem Matcher hinzufügen

In [10]:
pattern_test = [{'POS': 'NOUN'},
                {'POS': 'NOUN'}]

pattern_exmaple = [{'POS': 'ADJ', 'OP': '?'},
                   {'OP': '+', 'POS': 'NOUN'},
                   {'POS': 'NOUN', 'OP': '?'}]

In [11]:
matcher.add("match_test", [pattern_test])
matcher.add("match_example", [pattern_exmaple])

In [12]:
count_vectorizer = CountVectorizer(stop_words='english',ngram_range = (1,2))
counts = count_vectorizer.fit_transform(data['abstract'].values.astype('U'))
#print(counts)
#print(count_vectorizer.get_feature_names())
transformer = TfidfTransformer(smooth_idf=False)
tfidf = transformer.fit_transform(counts)

doc_term_matrix = tfidf.todense()
df = pd.DataFrame(doc_term_matrix, columns=count_vectorizer.get_feature_names())

In [13]:
for columns in data.itertuples():
    text = columns[3] #1 = title 2 = year 3 = abstract
    # Verarbeite den Text
    if type(text) is str:
            doc = nlp(text)
    
    matches = matcher(doc)
    sp  = [doc[match[1]:match[2]] for match in matches]
    filtered = filter_spans(sp)
    with doc.retokenize() as retokenizer:
        for s in filtered:
            retokenizer.merge(s, attrs = {"POS": "NOUN"})
    [(token.pos_, token.text) for token in doc]

    for token in doc:
        # Greife auf den Text, die Wortart und die Dependenzrelation des Tokens zu
        token_text = token.text
        token_pos = token.pos_
        token_dep = token.dep_
        token_ent = token.ent_type_

        if token.pos == NOUN:
            if not token.is_stop:
                words[token_text] += 1
                                          

## Wörter darstellen

In [14]:
print(type(words.most_common(100)))
for word,count in words.most_common(100):
    #print(f"{word}, {count}")
    print(df[word])

<class 'list'>
0       0.000000
1       0.000000
2       0.000000
3       0.053106
4       0.000000
          ...   
2160    0.000000
2161    0.000000
2162    0.000000
2163    0.000000
2164    0.000000
Name: paper, Length: 2165, dtype: float64
0       0.046872
1       0.000000
2       0.000000
3       0.000000
4       0.000000
          ...   
2160    0.000000
2161    0.000000
2162    0.000000
2163    0.000000
2164    0.000000
Name: approach, Length: 2165, dtype: float64
0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
2160    0.0
2161    0.0
2162    0.0
2163    0.0
2164    0.0
Name: developers, Length: 2165, dtype: float64
0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
2160    0.0
2161    0.0
2162    0.0
2163    0.0
2164    0.0
Name: results, Length: 2165, dtype: float64
0       0.054967
1       0.060432
2       0.056807
3       0.034051
4       0.059893
          ...   
2160    0.000000
2161    0.105792
2162    0.096523
2163    0.080046

KeyError: 'system'

## Liste erstellen

In [ ]:
l = list(words.most_common(30))
wordlist = []
for x in l:
    wordlist.append(x[0])


In [ ]:
words = pd.Series(wordlist,dtype=str)

In [ ]:
abstract = CreateTermDocMatrix('abstract')

In [ ]:
b = abstract[abstract.index.isin(words)]

In [ ]:
b.head()

## Balkendiagramm erstellen

In [ ]:
import altair as alt
import pandas as pd

source = pd.DataFrame({'Technologies': b.index,
                       'Anzahl': b[2020]})



alt.Chart(source).mark_bar().encode(
    x='Technologies',
    y='Anzahl'
)


In [ ]:
print(cosine_similarity(b))